In [ ]:
# làm sạch cơ bản

import pandas as pd
import numpy as np
from IPython.display import display # thư viện hỗ trợ hiển thị cho pandas

df = pd.read_csv("JobsDataset.csv") # đọc file

df.drop_duplicates(inplace=True) # xóa dòng bị trùng 100%
df.dropna(subset=['Query', 'Job Title', 'Description'], inplace=True) # xóa dòng nếu thiếu 1 trong 3 cột

In [3]:
print(df.head(20))

    ID           Query                             Job Title  \
0    1  Data Scientist  Junior Data Scientist Apprenticeship   
1    2  Data Scientist   HBO Data Scientist, Content Science   
2    3  Data Scientist                 Junior Data Scientist   
3    4  Data Scientist                     Jr Data Scientist   
4    5  Data Scientist       Data Scientist, Premium Content   
5    6  Data Scientist                        Data Scientist   
6    7  Data Scientist               Customer Data Scientist   
7    8  Data Scientist         Data Analyst / Data Scientist   
8    9  Data Scientist                        Data Scientist   
9   10  Data Scientist                 Junior Data Scientist   
10  11  Data Scientist                        Data Scientist   
11  12  Data Scientist             Geospatial Data Scientist   
12  13  Data Scientist              Associate Data Scientist   
13  14  Data Scientist                        Data Scientist   
14  15  Data Scientist                  